<a href="https://colab.research.google.com/github/andreyppsc/tensorflow-notebooks/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover the fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using &tf.function
* Using GPUs
* Exercises

## Introduction to tensors

In [3]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.7.0


In [4]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
# Check the number of dimensions of a tensor (ndim)
scalar.ndim

0

In [6]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [7]:
# Check the dimension of the vector
vector.ndim

1

In [8]:
# Create a matrix (has more than 1 dim)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [9]:
# Check the dim of the matrix
matrix.ndim

2

In [10]:
# Create another matrix
another_matrix = tf.constant([[10., .7],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10. ,  0.7],
       [ 3. ,  2. ],
       [ 8. ,  9. ]], dtype=float16)>

In [11]:
another_matrix.ndim

2

In [12]:
# Create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [13]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (ex wind speed and director)
* Matrix: a 2-dim array of numbers
* Tesnor: an _**n**_-dim array of numbers (where _**n**_ can be any number)

### Creating tensors with `tf.Variable`

In [14]:
# Create the same tensor with tf.Variable as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [16]:
# Let's try to change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: ignored

In [17]:
# How about we try .assing()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [18]:
# Let's try change our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: ignored

### Creating random tensors

Random tensors are tensors of some arbitrary size which 

In [19]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproductibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [20]:
not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### Other wasy to make tensors

In [21]:
tf.ones([10 ,7]), tf.zeros(shape=(3, 4))

(<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
 array([[1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)>)

### Turn NumPy arrays into tensors

In [22]:
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [23]:
A = tf.constant(numpy_A, shape=(3, 8))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [24]:
A.ndim

2

### Getting information from tensors

When dealing with tensors you will have to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [25]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [26]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions:", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions: 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [27]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [28]:
# Get the first element each dimension from each index except for the final one
rank_4_tensor[1:, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [29]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[4, 5],
                             [2, 1]])

In [30]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1] 

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 1], dtype=int32)>

In [31]:
# Add in extra dim to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[4],
        [5]],

       [[2],
        [1]]], dtype=int32)>

In [32]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[4],
        [5]],

       [[2],
        [1]]], dtype=int32)>

### Manipulating tensors

**Basic operations**

`+`, `-`, `*`, `/`

In [33]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])

tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [34]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [35]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [36]:
# Substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [37]:
# Division
tensor / 10

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [38]:
# We can use the tensordlow builtin function too
tf.multiply(tensor, 10), tf.add(tensor, 10)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [ 30,  40]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[20, 17],
        [13, 14]], dtype=int32)>)

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are **two** rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the inner dimmensions

In [39]:
# Matrix multiplication in tensor flow
print(tensor)

tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [40]:
tensor * tensor # looks like it's element wise

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [41]:
tensor_X = tf.constant([[1, 2],
                        [7, 2],
                        [3, 3]])
tensor_Y = tf.constant([[3, 5],
                        [6, 7],
                        [1, 8]])

tf.matmul(tensor_X, tensor_Y)

InvalidArgumentError: ignored

In [ ]:
# Let's change the shape of Y
tf.reshape(tensor_Y, (2, 3))

In [42]:
tensor_X @ tf.reshape(tensor_Y, (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[17,  7, 22],
       [35, 37, 58],
       [30, 18, 42]], dtype=int32)>

In [43]:
tf.matmul(tensor_X, tf.reshape(tensor_Y, (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[17,  7, 22],
       [35, 37, 58],
       [30, 18, 42]], dtype=int32)>

In [44]:
tf.matmul(tf.reshape(tensor_X, (2, 3)), tensor_Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[22, 75],
       [27, 55]], dtype=int32)>

In [45]:
# Can do the same with transpose
tf.transpose(tensor_X), tf.reshape(tensor_X, (2,3))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 7, 3],
        [2, 2, 3]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 7],
        [2, 3, 3]], dtype=int32)>)

In [46]:
tf.matmul(tf.transpose(tensor_X), tensor_Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[48, 78],
       [21, 48]], dtype=int32)>

 ***The dot product***
 
Matrix multiplication is also referred as the dot product.

You can perform matrix multiplication using:
* `tf.matmul`
* `tf.tensordot`
* `@` operator - python related

In [47]:
X = tensor_X
Y = tensor_Y

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [7, 2],
        [3, 3]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[3, 5],
        [6, 7],
        [1, 8]], dtype=int32)>)

In [48]:
# Using tf.tensordot
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[48, 78],
       [21, 48]], dtype=int32)>

In [49]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[13, 20, 17],
       [31, 56, 23],
       [24, 39, 27]], dtype=int32)>

In [50]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[17,  7, 22],
       [35, 37, 58],
       [30, 18, 42]], dtype=int32)>

In [51]:
# Check the values of Y, reshaped Y and transposed Y
print("Normal Y:")
print(Y, "\n")

print("Reshaped Y:")
print(tf.reshape(Y, (2, 3)), "\n")

print("Transposed Y:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[3 5]
 [6 7]
 [1 8]], shape=(3, 2), dtype=int32) 

Reshaped Y:
tf.Tensor(
[[3 5 6]
 [7 1 8]], shape=(2, 3), dtype=int32) 

Transposed Y:
tf.Tensor(
[[3 6 1]
 [5 7 8]], shape=(2, 3), dtype=int32)


Generally, when performing matrix multiplacation on two tensors and one of the axes doesn't line up, you will transpose rather than reshape one of the tensors

### Change the datatype of a tensor

In [52]:
# Create a new tensor with a default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [53]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [54]:
# Change from float32 to float16
B = tf.cast(B, tf.float16)
B.dtype

tf.float16

In [55]:
# Change from int32 to int16
C = tf.cast(C, tf.int16)
C.dtype

tf.int16

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [56]:
# Get the absolute values
D = tf.constant([-7, -10])
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Forms of aggregation:
* minimum
* maximum
* mean
* sum

In [57]:
# Create a random tensor with values between 0 and 100 pf size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([81, 80, 96, 95,  8, 87, 36,  5, 91, 58, 86, 74, 10, 92, 59, 89, 37,
       57, 63, 22, 35, 30,  1, 72,  2,  5, 24, 32, 50, 86, 25, 88, 26, 69,
       51, 46, 30, 76, 50,  2, 68, 27, 83, 99, 29, 98, 44, 51, 42, 73])>

In [58]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [59]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [60]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [61]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=52>

In [62]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2640>

In [63]:
# Find the variance
tf.math.reduce_variance(tf.cast(E, tf.complex64))

<tf.Tensor: shape=(), dtype=float32, numpy=899.48>

In [64]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, tf.complex64))

<tf.Tensor: shape=(), dtype=float32, numpy=29.991331>

In [65]:
# Find the variance using tfp
import tensorflow_probability as tfp

In [66]:
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=900>

In [67]:
tf.math.reduce_std(tf.cast(E, tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=29.991331>

In [68]:
custom_tensor = tf.constant(np.random.rand(3, 2), tf.float32)
custom_tensor, tf.matmul(custom_tensor, tf.transpose(custom_tensor)), tf.math.reduce_std(custom_tensor).numpy()

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.43794042, 0.740282  ],
        [0.957752  , 0.53567857],
        [0.59847236, 0.04979776]], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[0.7398093 , 0.8159915 , 0.2989596 ],
        [0.8159915 , 1.2042404 , 0.59986365],
        [0.2989596 , 0.59986365, 0.360649  ]], dtype=float32)>,
 0.27920035)

In [69]:
tf.math.reduce_variance(custom_tensor)

<tf.Tensor: shape=(), dtype=float32, numpy=0.07795284>

### Find the positional max and min of a tensor

In [70]:
tf.argmin(custom_tensor), tf.argmax(custom_tensor)

(<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 2])>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 0])>)

In [71]:
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [72]:
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [73]:
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [74]:
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [75]:
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [76]:
assert F[tf.argmin(F)] == tf.reduce_min(F)

### Squeezing a tensor

In [77]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [78]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [79]:
G_squeezed = tf.squeeze(G)
G_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

### One-hot encoding tensors

In [80]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red geen blue purple

# One-hot encode
tf.one_hot(some_list, 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [81]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, 4, "Yes", "No")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Yes', b'No', b'No', b'No'],
       [b'No', b'Yes', b'No', b'No'],
       [b'No', b'No', b'Yes', b'No'],
       [b'No', b'No', b'No', b'Yes']], dtype=object)>

### Squaring, log and square root

In [82]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [83]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [84]:
# Find the square root
tf.sqrt(tf.cast(H, tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.99999994, 1.4142134 , 1.7320508 , 1.9999999 , 2.236068  ,
       2.4494896 , 2.6457512 , 2.8284268 , 3.        ], dtype=float32)>

In [85]:
# Find the log
tf.math.log(tf.cast(H, tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beatutifully with NumPy arrays

In [86]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [87]:
# Convert our tensor back to our numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [88]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [89]:
# The default types of each are sightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the datatype of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [90]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

### Exercises

1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
2. Find the shape, rank and size of the tensors you created in 1.
3. Create two tensors containing random values between 0 and 1 with shape [5, 300].
4. Multiply the two tensors you created in 3 using matrix multiplication.
5. Multiply the two tensors you created in 3 using dot product.
6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
7. Find the min and max values of the tensor you created in 6 along the first axis.
8. Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
10. One-hot encode the tensor you created in 9.

In [100]:
# 1
ex_scalar = tf.constant(1)
ex_vector = tf.constant([1, 2, 3])
ex_matrix = tf.constant([[1, 2],
                         [2, 3]])
ex_tensor = tf.constant([[[1, 2],
                          [3, 4]],
                         [[5, 6],
                          [7, 8]]])
ex_scalar, ex_vector, ex_matrix, ex_tensor

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [2, 3]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
 array([[[1, 2],
         [3, 4]],
 
        [[5, 6],
         [7, 8]]], dtype=int32)>)

In [101]:
# 2 a)
ex_scalar.shape, ex_scalar.ndim, tf.size(ex_scalar)

(TensorShape([]), 0, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [102]:
# 2 b)
ex_vector.shape, ex_vector.ndim, tf.size(ex_vector)

(TensorShape([3]), 1, <tf.Tensor: shape=(), dtype=int32, numpy=3>)

In [103]:
# 2 c)
ex_matrix.shape, ex_matrix.ndim, tf.size(ex_matrix)

(TensorShape([2, 2]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=4>)

In [104]:
# 2 d)
ex_tensor.shape, ex_tensor.ndim, tf.size(ex_tensor)

(TensorShape([2, 2, 2]), 3, <tf.Tensor: shape=(), dtype=int32, numpy=8>)

In [107]:
# 3
ex_tensor_A = tf.random.uniform([5, 300], 0, 1)
ex_tensor_B = tf.random.uniform([5, 300], 0, 1)
ex_tensor_A, ex_tensor_B

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.7402308 , 0.33938193, 0.5692506 , ..., 0.08656633, 0.19666708,
         0.7110305 ],
        [0.13257599, 0.81292987, 0.01284087, ..., 0.02695906, 0.2927854 ,
         0.06227863],
        [0.282601  , 0.47868013, 0.77569485, ..., 0.28239357, 0.632112  ,
         0.44452012],
        [0.02029276, 0.7841259 , 0.16919017, ..., 0.26143157, 0.43055534,
         0.47421408],
        [0.2616079 , 0.00913954, 0.20257103, ..., 0.3548187 , 0.9939178 ,
         0.35239196]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.803156  , 0.49777734, 0.37054038, ..., 0.42893624, 0.49179256,
         0.39297235],
        [0.14517438, 0.72666824, 0.80103934, ..., 0.832801  , 0.649397  ,
         0.29312778],
        [0.4216962 , 0.6768439 , 0.5627917 , ..., 0.56929183, 0.90308595,
         0.31530714],
        [0.6198498 , 0.70206463, 0.994668  , ..., 0.03745186, 0.23192072,
         0.23984468],
        [0.01790

In [110]:
# 4
tf.matmul(ex_tensor_A, tf.transpose(ex_tensor_B))

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.5945208 , 0.16893664, 0.21093033, ..., 0.03713144, 0.09671941,
         0.2794153 ],
        [0.01924664, 0.5907303 , 0.01028604, ..., 0.02245153, 0.19013397,
         0.0182556 ],
        [0.11917176, 0.32399172, 0.43655464, ..., 0.16076435, 0.5708515 ,
         0.14016037],
        [0.01257846, 0.55050707, 0.16828805, ..., 0.0097911 , 0.09985471,
         0.11373772],
        [0.00468434, 0.00370438, 0.10878008, ..., 0.22244708, 0.47596633,
         0.08945112]], dtype=float32)>,
 <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[72.34169 , 73.843315, 71.9973  , 69.78642 , 68.598114],
        [76.2998  , 75.77642 , 71.655014, 72.104355, 75.299416],
        [72.80916 , 75.74102 , 71.199   , 73.046555, 71.71448 ],
        [74.654816, 76.64436 , 73.32848 , 72.2124  , 73.30908 ],
        [77.44372 , 78.682274, 71.98375 , 72.70094 , 72.917274]],
       dtype=float32)>)

In [116]:
# 5
tf.tensordot(ex_tensor_A, tf.reshape(ex_tensor_B, shape=(300, 5)), 1)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[73.85136 , 75.33441 , 74.32788 , 73.18034 , 68.09682 ],
       [73.48224 , 76.2337  , 76.62964 , 75.73997 , 72.134995],
       [72.03822 , 76.3349  , 76.72305 , 73.221634, 70.411995],
       [74.998825, 76.65549 , 76.49006 , 74.25154 , 71.491   ],
       [73.049706, 76.65055 , 74.62923 , 76.25304 , 73.068634]],
      dtype=float32)>

In [ ]:
# 6 shape = (224,224,3)
ex_tensor_C = tf.random.uniform((224, 224, 3), 0, 1)
ex_tensor_C

In [ ]:
# 7
tf.reduce_max(ex_tensor_C, 0), tf.reduce_min(ex_tensor_C, 0)

In [ ]:
# 8 Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
ex_tensor_D = tf.random.uniform(shape=(1, 224, 224, 3))
tf.squeeze(ex_tensor_D)

In [135]:
# 9 Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
ex_tensor_E = tf.constant([24, 2, 57, 19, 20, 6, 89, 3, 5, 10])
ex_tensor_E, tf.argmax(ex_tensor_E).numpy()

(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([24,  2, 57, 19, 20,  6, 89,  3,  5, 10], dtype=int32)>,
 6)

In [137]:
# 10 One-hot encode the tensor you created in 9.
tf.one_hot(ex_tensor_E, 10)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>